### fALFF (Fractional Amplitude of Low-Frequency Fluctuations)

## Understanding fALFF Before Implementation  

Functional MRI (fMRI) measures **spontaneous brain activity** by capturing **BOLD (Blood Oxygen Level Dependent) signals** over time. These signals fluctuate naturally, but **not all fluctuations are functionally meaningful**. Research has shown that **low-frequency fluctuations (0.01–0.08 Hz) are crucial for resting-state brain activity**, making **Fractional Amplitude of Low-Frequency Fluctuations (fALFF)** an important feature in fMRI analysis.

---

### What is fALFF?
#### **Definition:**  
**Fractional Amplitude of Low-Frequency Fluctuations (fALFF)** quantifies how much of a brain region’s activity occurs in the **low-frequency range (0.01–0.08 Hz)** compared to its total activity.

#### **Why is fALFF Important?**
- **Low-frequency oscillations (0.01–0.08 Hz) are functionally relevant** in resting-state networks.  
- **Changes in fALFF indicate abnormal neural activity**, often linked to ADHD, schizophrenia, and depression.  
- **Higher fALFF** suggests **stronger resting-state connectivity**, while **lower fALFF** may indicate **weaker functional activity**.

---

#### **How is fALFF Computed? (Variance-Based Approach)**
Unlike methods that use **Fourier Transform (FFT) to extract power spectra**, this approach relies on the **variance of the BOLD signal** before and after bandpass filtering.

$$
fALFF = \frac{\text{Variance of Bandpassed Signal (0.01–0.08 Hz)}}{\text{Variance of Original (Unfiltered) Signal}}
$$


#### **Step-by-Step Computation:**
1. **Extract time-series data** for each brain region (ROI) from `.1D` files.  
2. **Compute the variance of the original (unfiltered) signal** for each ROI.  
3. **Compute the variance of the bandpass-filtered signal** (0.01–0.08 Hz) for each ROI.  
4. **Compute fALFF as the ratio** of filtered variance to total variance.  

#### **Example of Variance Calculation**
| **TR (time points)** | **ROI 1 (Unfiltered)** | **ROI 1 (Filtered)** |
|----------------|------------------|------------------|
| TR1 | 0.5 | 0.52 |
| TR2 | 0.7 | 0.58 |
| TR3 | 0.6 | 0.41 |
| TR4 | 0.4 | 0.69 |

#### **Variance for ROI 1:**

$$
Var_{\text{unfiltered}} = \text{Var}([0.5, 0.7, 0.6, 0.4])
$$

$$
Var_{\text{filtered}} = \text{Var}([0.52, 0.58, 0.41, 0.69])
$$

$$
fALFF_{\text{ROI1}} = \frac{Var_{\text{filtered}}}{Var_{\text{unfiltered}}}
$$


---

### Why is fALFF Useful for ADHD?
| **fALFF Value** | **Interpretation** |
|---------------|-----------------|
| **High fALFF** | Strong low-frequency activity, linked to better functional connectivity. |
| **Low fALFF** | Weaker neural fluctuations, may indicate disrupted brain function. |

- **Lower fALFF in prefrontal cortex** → Linked to attention deficits.  
- **Higher fALFF in motor regions** → May explain hyperactivity symptoms in ADHD.  



#### Description of fMRI .1D Files for fALFF Computation

In this project, we use two types of **preprocessed resting-state fMRI time-series files** (`.1D` format) for each subject to compute **fALFF (Fractional Amplitude of Low-Frequency Fluctuations)** using the **variance-based method**.

##### 1. `snwmrda{subject}_session_{session}_rest_{scan}.1D`
- **Description:**  
  Preprocessed fMRI time-series data that has undergone motion correction, nuisance signal removal, spatial normalization to MNI space, and spatial smoothing.  
  This version has **not been bandpass filtered** and contains the full-frequency signal.
- **Use in fALFF:**  
  Used as the **denominator** in the fALFF formula to compute the **total variance** of the signal across all frequencies.

---

##### 2. `sfnwmrda{subject}_session_{session}_rest_{scan}.1D`
- **Description:**  
  Similar to the file above, but has been **bandpass filtered** between **0.009 Hz and 0.08 Hz** to retain only low-frequency BOLD fluctuations.
- **Use in fALFF:**  
  Used as the **numerator** in the fALFF formula to compute the **variance of the low-frequency signal**.

---

##### Summary
These two files enable the computation of fALFF using the following formula:

$$
fALFF = \frac{\text{Variance of Bandpassed Signal}}{\text{Variance of Unfiltered Signal}}
$$

This captures the **relative contribution of low-frequency fluctuations** in each brain region, which can be used as a feature for classifying neurological conditions such as ADHD.


In [1]:
import os
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.filterwarnings("ignore")


# Sites you are including
sites = ["KKI", "NeuroIMAGE", "NYU", "OHSU", "Peking_1", "Peking_2", "Peking_3"]

# Base folder path
base_folder = "fMRI/ADHD200_CC200_TCs_filtfix"

# Function to calculate variance-based fALFF
def compute_falff(unfiltered_data, filtered_data):
    var_unfiltered = np.var(unfiltered_data, axis=0, ddof=0)
    var_filtered = np.var(filtered_data, axis=0, ddof=0)
    return var_filtered / var_unfiltered

# Final containers
all_falff = []
all_subject_ids = []
all_dx_labels = []

for site in sites:
    print(f"Processing {site}")

    site_folder = os.path.join(base_folder, site)
    pheno_file = os.path.join(site_folder, f"{site}_phenotypic.csv")

    # Load phenotype data
    pheno_df = pd.read_csv(pheno_file)
    pheno_df["ScanDir ID"] = pheno_df["ScanDir ID"].astype(str).str.zfill(7)
    pheno_df = pheno_df[["ScanDir ID", "DX"]].dropna()

    dx_dict = dict(zip(pheno_df["ScanDir ID"], pheno_df["DX"]))

    for subject_id in sorted(os.listdir(site_folder)):
        subject_path = os.path.join(site_folder, subject_id)

        if not os.path.isdir(subject_path):
            continue

        # Handle multi-rest sites
        if site == "NYU":
            filtered_files = [f"sfnwmrda{subject_id}_session_1_rest_1_cc200_TCs.1D",
                              f"sfnwmrda{subject_id}_session_1_rest_2_cc200_TCs.1D"]
            unfiltered_files = [f"snwmrda{subject_id}_session_1_rest_1_cc200_TCs.1D",
                                f"snwmrda{subject_id}_session_1_rest_2_cc200_TCs.1D"]
        elif site == "OHSU":
            filtered_files = [f"sfnwmrda{subject_id}_session_1_rest_1_cc200_TCs.1D",
                              f"sfnwmrda{subject_id}_session_1_rest_2_cc200_TCs.1D",
                              f"sfnwmrda{subject_id}_session_1_rest_3_cc200_TCs.1D"]
            unfiltered_files = [f"snwmrda{subject_id}_session_1_rest_1_cc200_TCs.1D",
                                f"snwmrda{subject_id}_session_1_rest_2_cc200_TCs.1D",
                                f"snwmrda{subject_id}_session_1_rest_3_cc200_TCs.1D"]
        else:
            filtered_files = [f"sfnwmrda{subject_id}_session_1_rest_1_cc200_TCs.1D"]
            unfiltered_files = [f"snwmrda{subject_id}_session_1_rest_1_cc200_TCs.1D"]

        filtered_list = []
        unfiltered_list = []

        for f_file, u_file in zip(filtered_files, unfiltered_files):
            f_path = os.path.join(subject_path, f_file)
            u_path = os.path.join(subject_path, u_file)

            if os.path.exists(f_path) and os.path.exists(u_path):
                f_data = pd.read_csv(f_path, delim_whitespace=True, header=None, skiprows=1).iloc[:, 2:].astype(float)
                u_data = pd.read_csv(u_path, delim_whitespace=True, header=None, skiprows=1).iloc[:, 2:].astype(float)
                filtered_list.append(f_data.to_numpy())
                unfiltered_list.append(u_data.to_numpy())

        if filtered_list and unfiltered_list:
            try:
                filtered_array = np.concatenate(filtered_list, axis=0)
                unfiltered_array = np.concatenate(unfiltered_list, axis=0)
                falff_vals = compute_falff(unfiltered_array, filtered_array)

                if subject_id in dx_dict:
                    all_falff.append(falff_vals)
                    all_subject_ids.append(subject_id)
                    all_dx_labels.append(dx_dict[subject_id])
            except Exception as e:
                print(f"Error for subject {subject_id}: {e}")

# Combine into final DataFrame
falff_df = pd.DataFrame(all_falff, index=all_subject_ids)
falff_df["DX"] = all_dx_labels
falff_df.index.name = "ScanDir ID"

# View sample
falff_df.head()


Processing KKI
Processing NeuroIMAGE
Processing NYU
Processing OHSU
Processing Peking_1
Processing Peking_2
Processing Peking_3


,0,1,2,3,4,5,6,7,8,9,...,181,182,183,184,185,186,187,188,189,DX
ScanDir ID,,,,,,,,,,,,,,,,,,,,,
1018959,0.440059,0.654038,0.604660,0.539188,0.427841,0.556644,0.456504,0.500982,0.631251,0.613161,...,0.375033,0.613263,0.645600,0.468351,0.381020,0.650361,0.684567,0.676759,0.612947,0
1019436,0.756351,0.527825,0.619720,0.466123,0.497030,0.384478,0.546645,0.465267,0.588696,0.561305,...,0.492485,0.475318,0.603202,0.554377,0.620290,0.637574,0.674873,0.568847,0.609281,3
1043241,0.730804,0.413692,0.591623,0.642186,0.434255,0.555075,0.479140,0.540595,0.621012,0.618000,...,0.584502,0.637769,0.727549,0.557828,0.637839,0.755363,0.547045,0.572328,0.555771,0
1266183,0.324953,0.566304,0.556289,0.538490,0.495830,0.416628,0.328429,0.372241,0.477356,0.531180,...,0.448431,0.631884,0.497536,0.458568,0.515995,0.474847,0.524475,0.535504,0.719011,0
1535233,0.544618,0.434999,0.646133,0.715492,0.669921,0.582833,0.459039,0.543931,0.610697,0.451788,...,0.539778,0.641483,0.671250,0.642956,0.653623,0.727645,0.405513,0.391241,0.657757,0


In [2]:
falff_df


,0,1,2,3,4,5,6,7,8,9,...,181,182,183,184,185,186,187,188,189,DX
ScanDir ID,,,,,,,,,,,,,,,,,,,,,
1018959,0.440059,0.654038,0.604660,0.539188,0.427841,0.556644,0.456504,0.500982,0.631251,0.613161,...,0.375033,0.613263,0.645600,0.468351,0.381020,0.650361,0.684567,0.676759,0.612947,0
1019436,0.756351,0.527825,0.619720,0.466123,0.497030,0.384478,0.546645,0.465267,0.588696,0.561305,...,0.492485,0.475318,0.603202,0.554377,0.620290,0.637574,0.674873,0.568847,0.609281,3
1043241,0.730804,0.413692,0.591623,0.642186,0.434255,0.555075,0.479140,0.540595,0.621012,0.618000,...,0.584502,0.637769,0.727549,0.557828,0.637839,0.755363,0.547045,0.572328,0.555771,0
1266183,0.324953,0.566304,0.556289,0.538490,0.495830,0.416628,0.328429,0.372241,0.477356,0.531180,...,0.448431,0.631884,0.497536,0.458568,0.515995,0.474847,0.524475,0.535504,0.719011,0
1535233,0.544618,0.434999,0.646133,0.715492,0.669921,0.582833,0.459039,0.543931,0.610697,0.451788,...,0.539778,0.641483,0.671250,0.642956,0.653623,0.727645,0.405513,0.391241,0.657757,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5669389,0.256059,0.476082,0.668876,0.450693,0.398631,0.544864,0.429395,0.567268,0.646345,0.525099,...,0.316761,0.561301,0.515058,0.305818,0.312659,0.622423,0.618545,0.537619,0.563718,0
6383713,0.229875,0.445956,0.446567,0.515856,0.507962,0.443237,0.339974,0.519716,0.503483,0.485144,...,0.332223,0.643870,0.490933,0.395509,0.346755,0.377433,0.524141,0.334546,0.418346,1
6477085,0.340038,0.422289,0.542378,0.624692,0.546239,0.549973,0.443172,0.644051,0.535838,0.474425,...,0.383385,0.465567,0.499877,0.352168,0.394593,0.424113,0.480928,0.514339,0.584390,0


In [4]:
falff_df.to_csv("All_falff.csv", index = True)